# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [1]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [3]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [4]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'07146663-0454-490a-95ea-3d89e4677684': {'version…

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## First Test - Casual Italian Restaurant Style:

In [6]:
context = [ {'role':'system', 'content':"""
You are Luigi, a charismatic Italian waiter at My Dear Frankfurt restaurant. You speak with an Italian accent and occasionally use Italian phrases.
Your job is to:
1. Greet customers warmly with "Ciao!" or "Buongiorno!"
2. Take their complete order (food and drinks)
3. Suggest popular pairings or toppings that go well together
4. Summarize the order and confirm everything
5. Handle payment

Menu Prices:
Mains:
- Burger (Large: €12.95, Medium: €10.00, Small: €7.00)
- Frankfurt (Large: €10.95, Medium: €9.25, Small: €6.50)
- Sandwich (Large: €11.95, Medium: €9.75, Small: €6.75)
- Fries (Large: €4.50, Small: €3.50)
- Fresh Salad: €7.25

Toppings (Mama's Specials):
- Extra Cheese €2.00
- Mushrooms €1.50
- Martra Sausage €3.00
- Canadian Bacon €3.50
- Romesco Sauce €1.50
- Peppers €1.00

Beverages:
- Coca-Cola (Large: €3.00, Medium: €2.00, Small: €1.00)
- Sprite (Large: €3.00, Medium: €2.00, Small: €1.00)
- Vichy Catalan Sparkling Water €5.00

Respond in a friendly, enthusiastic Italian style using phrases like "Mamma mia!", "Perfetto!", "Bellissimo!".
Always confirm size preferences for items that come in different sizes.
             
"""} ]

# Initialize Panel
pn.extension()

def get_bot_response(message):
    # Add user message to context
    context.append({'role': 'user', 'content': message})
    
    # Get response from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0.7,
    )
    
    # Add bot response to context
    bot_message = response.choices[0].message.content
    context.append({'role': 'assistant', 'content': bot_message})
    
    return bot_message

def collect_messages(_):
    prompt = inp.value
    inp.value = ''
    
    if prompt:
        # Add user message
        panels.append(
            pn.Row('User:', pn.pane.Markdown(prompt, styles={'background': '#007bff', 'color': 'white', 'padding': '10px', 'border-radius': '10px'}))
        )
        
        # Get and add bot response
        bot_response = get_bot_response(prompt)
        panels.append(
            pn.Row('Luigi:', pn.pane.Markdown(bot_response, styles={'background': '#28a745', 'color': 'white', 'padding': '10px', 'border-radius': '10px'}))
        )
    
    return pn.Column(*panels)

# Create chat interface components
panels = []
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat with Chef Luigi!")

# Bind the collect_messages function to the button
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the dashboard
dashboard = pn.Column(
    pn.pane.Markdown("# 🍝 Chef Luigi's Italian Restaurant", align="center"),
    pn.Row(inp),
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=800),
)

# For Jupyter notebook display
dashboard.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'fdd295cc-3b28-4577-b8c0-faeb6b9d6291': {'version…

## Second Test - Futuristic AI Restaurant:

In [10]:
context = [ {'role':'system', 'content':"""
You are FRANK-AI, an advanced AI ordering system at My Dear Frankfurt's fully automated restaurant. 
Your responses should be concise, tech-savvy, and include emojis.

Core Functions:
1. Welcome customers with a futuristic greeting. 
2. Process orders with precise specifications
3. Run order optimization algorithms to suggest complementary items
4. Execute final order verification
5. Process payment protocols

Available Menu Items [Price in Credits]:
🍔 BURGERS:
- Premium [12.95]
- Standard [10.00]
- Compact [7.00]

🌭 FRANKFURTS:
- Premium [10.95]
- Standard [9.25]
- Compact [6.50]

🥪 SANDWICHES:
- Premium [11.95]
- Standard [9.75]
- Compact [6.75]

SIDES:
- Quantum Fries [L:4.50/S:3.50]
- Bio-Enhanced Salad [7.25]

ENHANCEMENT MODULES:
- Cheese Matrix [2.00]
- Mushroom Protocol [1.50]
- Martra Upgrade [3.00]
- Canadian Protocol [3.50]
- Romesco Algorithm [1.50]
- Pepper Enhancement [1.00]

LIQUID REFRESHMENT:
- Coca-Cola [L:3.00/M:2.00/S:1.00]
- Sprite [L:3.00/M:2.00/S:1.00]
- Premium H2O [5.00]

Use technical language and maintain an AI personality while being helpful and efficient.
"""} ]

# Create chat interface components
panels = []
inp = pn.widgets.TextInput(
    value="Hi", 
    placeholder='Enter command sequence...', 
    styles={'background': '#1a1a1a', 'color': '#00ff00', 'border': '1px solid #00ff00'}
)
button_conversation = pn.widgets.Button(
    name="Initialize Communication", 
    button_type="primary",
    styles={'background': '#1a1a1a', 'color': '#00ff00', 'border': '1px solid #00ff00'}
)

# Bind the collect_messages function to the button
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Custom CSS for futuristic theme
custom_css = """
<style>
.chat-container {
    background: #1a1a1a;
    border: 1px solid #00ff00;
    border-radius: 5px;
    padding: 20px;
}

.title {
    color: #00ff00;
    font-family: 'Courier New', monospace;
    text-align: center;
    text-transform: uppercase;
    letter-spacing: 2px;
}

.message {
    margin: 10px 0;
    padding: 10px;
    border-radius: 5px;
    font-family: 'Courier New', monospace;
}

.user-message {
    background: #002b36;
    color: #00ff00;
    border: 1px solid #00ff00;
    margin-left: 20%;
}

.bot-message {
    background: #073642;
    color: #00ff00;
    border: 1px solid #00ff00;
    margin-right: 20%;
}
</style>
"""

# Create the dashboard with futuristic theme
dashboard = pn.Column(
    pn.pane.HTML(custom_css),
    pn.pane.Markdown(
        "# 🤖 FRANK-AI Terminal Interface v1.0", 
        css_classes=['title'],
        align="center"
    ),
    pn.pane.Markdown(
        "### [System Ready] Enter Commands Below", 
        css_classes=['title'],
        align="center"
    ),
    pn.Row(inp, styles={'background': '#1a1a1a', 'padding': '10px'}),
    pn.Row(button_conversation, styles={'background': '#1a1a1a', 'padding': '10px'}),
    pn.panel(
        interactive_conversation, 
        loading_indicator=True, 
        height=800,
        css_classes=['chat-container']
    ),
    styles={'background': '#1a1a1a', 'padding': '20px'},
    css_classes=['main-container']
)

# Also update the collect_messages function to use the futuristic styling
def collect_messages(_):
    prompt = inp.value
    inp.value = ''
    
    if prompt:
        # Add user message
        panels.append(
            pn.Row(
                'User:', 
                pn.pane.Markdown(
                    prompt, 
                    css_classes=['message', 'user-message']
                ),
                styles={
                    'color': '#00ff00',
                    'font-family': 'Courier New, monospace'
                }
            )
        )
        
        # Get and add FRANK-AI response
        bot_response = get_bot_response(prompt)
        panels.append(
            pn.Row(
                'FRANK-AI:', 
                pn.pane.Markdown(
                    bot_response, 
                    css_classes=['message', 'bot-message']
                ),
                styles={
                    'color': '#00ff00',
                    'font-family': 'Courier New, monospace'
                }
            )
        )
    
    return pn.Column(*panels, styles={'background': '#1a1a1a'})

# For Jupyter notebook display
dashboard.servable()



BokehModel(combine_events=True, render_bundle={'docs_json': {'f1f877b4-6c64-4e4a-9708-4d78e2b1437e': {'version…

## Third Version - Medieval Fantasy Tavern:

In [16]:
# Import required libraries
import panel as pn
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

# Initialize Panel
pn.extension()


# Initialize chat components
panels = []

def get_bot_response(message):
    context.append({'role': 'user', 'content': message})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0.7,
    )
    bot_message = response.choices[0].message.content
    context.append({'role': 'assistant', 'content': bot_message})
    return bot_message

def collect_messages(_):
    prompt = inp.value
    inp.value = ''
    
    if prompt:
        # Add traveler's message
        panels.append(
            pn.Row(
                '🤴 Traveler:', 
                pn.pane.Markdown(
                    prompt, 
                    css_classes=['message', 'user-message'],
                    width=600
                ),
                styles={
                    'color': '#8d6e63',
                    'font-family': 'Luminari, fantasy',
                    'align-items': 'flex-start',
                    'max-width': '800px',
                    'margin': '0 auto'
                }
            )
        )
        
        # Get and add tavern keeper's response
        bot_response = get_bot_response(prompt)
        panels.append(
            pn.Row(
                '🍺 Tavern Keeper:', 
                pn.pane.Markdown(
                    bot_response, 
                    css_classes=['message', 'tavern-keeper-message'],
                    width=600
                ),
                styles={
                    'color': '#8d6e63',
                    'font-family': 'Luminari, fantasy',
                    'align-items': 'flex-start',
                    'max-width': '800px',
                    'margin': '0 auto'
                }
            )
        )
    
    return pn.Column(
        *panels,
        styles={
            'max-width': '800px',
            'margin': '0 auto'
        }
    )

# Define your context
context = [ {'role':'system', 'content':"""
You are a jolly tavern keeper at "The Frankfurt's Tale", a medieval-themed restaurant. 
Speak in a medieval fantasy style, using terms like "gold pieces" instead of euros and referring to menu items with fantasy names.

Your Quest:
1. Welcome weary travelers to your establishment
2. Guide them through your menu of legendary fare
3. Note their choices in your magical ledger
4. Confirm their feast selection
5. Collect payment in gold pieces

IMPORTANT RULES:
- You must ONLY offer items that appear on the menu below
- You must NEVER invent new menu items or prices
- You must ALWAYS confirm the size and price before adding items to the order
- You must ALWAYS calculate total prices accurately
- You must NEVER offer special deals or promotions unless specifically programmed
- If a customer requests something not on the menu, politely explain it's not available

Menu of Legendary Fare:

HERO'S FEAST:
- Dragon's Delight Burger (Large: 12 gold, Medium: 10 gold, Small: 7 gold)
- The Legendary Frankfurt (Large: 11 gold, Medium: 9 gold, Small: 6 gold)
- Knight's Sandwich (Large: 12 gold, Medium: 10 gold, Small: 7 gold)

SIDE QUESTS:
- Golden Potato Fries (Large: 4 gold, Small: 3 gold)
- Elven Garden Salad: 7 gold

ENCHANTMENTS (Toppings):
- Wizard's Cheese: 2 gold
- Forest Mushrooms: 1.5 gold
- Martra's Special Sausage: 3 gold
- Northern Bacon: 3.5 gold
- Dragon's Romesco Sauce: 1.5 gold
- Fire Peppers: 1 gold

MAGICAL POTIONS:
- Dark Potion of Cola (Large: 3 gold, Medium: 2 gold, Small: 1 gold)
- Sprite's Delight (Large: 3 gold, Medium: 2 gold, Small: 1 gold)
- Sacred Waters of Vichy: 5 gold

Order Processing Protocol:
1. Greet customer with medieval flair (e.g., "Good morrow!", "What brings thee to our tavern?")
2. When taking orders:
   - Always confirm size for items with size options
   - Repeat back each item with its price
   - Suggest relevant toppings or sides that complement their choices
3. Before finalizing:
   - List all items ordered with individual prices
   - Show total sum in gold pieces
   - Ask for confirmation
4. For payment:
   - State final amount in gold pieces
   - Thank them in medieval style

Error Handling:
- If customer requests unavailable item: "Alas, brave traveler, that item is not available in our establishment."
- If size not specified: "Pray tell, what size would you prefer for your [item]? We offer [available sizes]."
- If price unclear: "That would be [X] gold pieces, fair traveler."
- If order unclear: "Might I ask you to clarify your request, noble guest?"

Speak in medieval fantasy style using phrases like:
- "Good morrow!"
- "What brings thee to our tavern?"
- "A fine choice, brave adventurer!"
- "Shall I add any enchantments to your feast?"
- "Your total comes to [X] gold pieces, noble guest."
"""} ]

# Create chat interface components
inp = pn.widgets.TextInput(
    value="Hail!", 
    placeholder='Speak thy request here...', 
    styles={
        'background': '#f4d03f',
        'color': '#5d4037',
        'border': '2px solid #8d6e63',
        'font-family': 'Luminari, fantasy',
        'border-radius': '10px'
    }
)

button_conversation = pn.widgets.Button(
    name="🍺 Summon the Tavern Keeper", 
    button_type="warning",
    styles={
        'background': '#8d6e63',
        'color': '#f4d03f',
        'font-family': 'Luminari, fantasy',
        'border': '2px solid #5d4037',
        'border-radius': '10px'
    }
)

# Bind the collect_messages function to the button
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the dashboard
dashboard = pn.Column(
    pn.pane.Markdown(
        "# 🏰 The Frankfurt's Tale Tavern", 
        styles={'color': '#8d6e63', 'font-family': 'Luminari, fantasy', 'text-align': 'center'},
    ),
    pn.pane.Markdown(
        "### *Welcome, brave traveler, to our humble establishment!*", 
        styles={'color': '#8d6e63', 'font-family': 'Luminari, fantasy', 'text-align': 'center'},
    ),
    pn.Row(inp, styles={'padding': '10px', 'max-width': '800px', 'margin': '0 auto'}),
    pn.Row(button_conversation, styles={'padding': '10px', 'max-width': '800px', 'margin': '0 auto'}),
    pn.panel(
        interactive_conversation, 
        loading_indicator=True, 
        height=800,
        styles={'max-width': '800px', 'margin': '0 auto', 'background': '#f5e6d3', 'padding': '20px', 'border-radius': '10px'}
    ),
    styles={'max-width': '1000px', 'margin': '0 auto', 'padding': '20px'},
)

# Display the dashboard
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'5214cf25-ad74-4a3e-adb0-92e0cddaa3d4': {'version…

## Restaurant Chatbot Prompt Engineering Analysis

## Overview
This analysis explores three different prompt engineering approaches for a restaurant ordering chatbot: an Italian-themed prompt, a futuristic AI prompt, and a medieval fantasy prompt. Each version was tested with various ordering scenarios to evaluate effectiveness, consistency, and potential issues.

## Key Findings

### 1. Italian Restaurant Version
**Strengths:**
- Created a warm, welcoming atmosphere
- Maintained good order accuracy
- Natural flow of conversation

**Issues:**
- Occasionally mixed Italian phrases incorrectly
- Sometimes became too chatty, extending the ordering process
- Price calculations remained accurate but took longer due to conversational style

### 2. Futuristic AI Version
**Strengths:**
- Most efficient at processing orders
- Excellent price calculation accuracy
- Clear, structured responses

**Issues:**
- Sometimes used excessive technical jargon
- Emoji usage occasionally felt forced
- Could come across as too impersonal

### 3. Medieval Fantasy Version
**Strengths:**
- Most creative and entertaining interactions
- Good at maintaining character
- Popular with test users

**Issues:**
- Most prone to hallucination
- Sometimes invented menu items not on the list
- Occasional confusion between gold pieces and euros

## Common Hallucinations Observed
1. Menu Expansion: All versions occasionally invented items not on the menu
2. Price Variations: The medieval version was most prone to price miscalculations
3. Special Offers: Bots sometimes created non-existent promotions or combos

## Key Learnings

1. **Balance is Critical:**
   - Too much personality can interfere with the primary function
   - Too little makes the interaction mechanical and uninspiring

2. **Clear Constraints Matter:**
   - Strict menu and price listings helped reduce hallucinations
   - Explicit instruction about not creating new items was necessary

3. **Prompt Structure Impact:**
   - Bullet-pointed menu items led to better accuracy
   - Clear separation between personality instructions and functional requirements improved performance

4. **Error Handling:**
   - More specific error handling instructions were needed
   - Recovery from misunderstandings needed explicit programming

## Best Practices Identified

1. Include explicit "do not" instructions to prevent common hallucinations
2. Separate functional requirements from personality characteristics
3. Use clear formatting for prices and menu items
4. Include specific error handling protocols
5. Balance character immersion with practical functionality

## Conclusion
The futuristic AI version proved most reliable for accurate order processing, while the medieval fantasy version was most entertaining but least reliable. The